In [1]:
#Import necessary packages
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder,DeployTransactionBuilder,MessageTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
import requests
import json
import pickle
import csv
import os
from repeater import retry
from iconsdk.libs.in_memory_zip import gen_deploy_data_content

In [2]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
SCORE_ADDRESS = "cxf145f42657dbf2f98f02427b55e27c67d2951220"
NEWTON_ADDRESS = "hx9393939393939393939393939393939393939399"
DEPLOY_PARAMS = {}

In [3]:
#Start ICON service and wallet
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
#icon_service = IconService(HTTPProvider("http://localhost:9000", 3))
operator = KeyWallet.load('config/yeouido/keystores/operator', 'bgsa199@#')
prajwol = KeyWallet.load('config/yeouido/keystores/testPrajwol', 'bgsa199@#')



In [6]:
def test_method(method_name:str, params:dict):
    call_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method(method_name)\
    .params(params)\
    .build()

    estimate_step = icon_service.estimate_step(call_transaction)
    step_limit = estimate_step + 100000
    signed_transaction = SignedTransaction(call_transaction, operator,step_limit)

    tx_hash = icon_service.send_transaction(signed_transaction)
    return tx_hash

def transfer_transaction(to, value):
    test_method('transfer', {'_to':to, '_value':value})

def update(name, symbol, supply, decimals=18):
    UPDATE_PARAMS =  {
            "_tokenName": name,
            "_symbolName": symbol,
			"_initialSupply": supply,
            "_decimals": decimals
		}
        
    deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(UPDATE_PARAMS)\
    .build()

    estimate_step = icon_service.estimate_step(deploy_transaction)
    step_limit = estimate_step + 100000
    signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)    
    tx_hash = icon_service.send_transaction(signed_transaction)

def external_methods():
    external_methods = ["name", "symbol", "decimals", "totalSupply"]
    for method in external_methods:
        call_method(method)
    
def call_method(method_name, params={}):
    call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method_name)\
                    .params(params)\
                    .build()
    print(icon_service.call(call))



In [9]:
external_methods()

TestToken
TS
0x12
0x3635c9adc5dea00000


In [8]:
update('TestToken','TS',1000, 18)

